<a href="https://colab.research.google.com/github/Idaogah/datalab/blob/main/dynamic_SQL_table_creator_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This dynamic SQL table creator lets you create a procedure that generate table whose name, columns, and data types are not known until runtime. A dynamic table creator connect to the database and look out for the list of tables to be created and the required columns and their data types based on the business needs. The script also create schemas for the tables in the same group.

The script can as well delete unwanted schemas as specified. It can also update table values as required.

# Install Packages

In [ ]:
!pip install psycopg2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Import Libraries

In [ ]:
# libraries
import psycopg2
from google.colab import drive
drive.mount('/content/drive')

# Connect database

In [2]:
# # Provide your database credentials and uncomment relevant codelines
# # Connect to the database
# conn = psycopg2.connect(
#     dbname= ' ',
#     user= ' ',
#     password= ' ',
#     host= ' ',
#     port=
# )

# print("Connection string:", conn.get_dsn_parameters())


# Cursor to execute SQL commands

In [ ]:
# Create a cursor object to execute SQL commands
cur = conn.cursor()

# Get the financial_asset_symbol column values from the financial_asset_list table
cur.execute('SELECT financial_asset_symbol FROM "dySPP"."financial_asset_list"')
financial_asset_symbols = cur.fetchall()

columns = []
table_name = []
# Iterate over the financial_asset_symbols and create a schema for each one if it doesn't already exist
for symbol, in financial_asset_symbols:
  cur.execute(f"CREATE SCHEMA IF NOT EXISTS \"TEST_{symbol}\"")
  cur.execute(f'ALTER SCHEMA \"TEST_{symbol}\" OWNER TO "godwin.ogah"')

  # Query the table_list_view to get table names
  cur.execute('SELECT table_names FROM "dySPP".table_list_view')
  table_names = cur.fetchall()

  # Iterate through each table name and create table
  for table_name in table_names:
    table_name = table_name[0]
    # print("TABLE NAME: ", table_name)

    # Create the table
    cur.execute(f"CREATE TABLE IF NOT EXISTS \"TEST_{symbol}\".\"{table_name}\" (id SERIAL PRIMARY KEY)")
    cur.execute(f'ALTER TABLE \"TEST_{symbol}\".\"{table_name}\" OWNER TO "godwin.ogah"')

    # Query the table_list_column_view to get the columns and data types for the table
    cur.execute(f'SELECT table_view_name, column_name, postgresql_data_type_name FROM "dySPP".table_list_view_columns') # WHERE table_view_name = \'{table_name}\'
    columns = cur.fetchall()
    # print("Table: ", table_name)

    # Iterate through each column and add it to the table
    for col1, col2, col3 in columns:
      if col1 == table_name:
          column_name = col1 + "_" + col2
          data_type = col3
          # print("COLUMNS: ", column_name)
          try:
            cur.execute(f"ALTER TABLE \"TEST_{symbol}\".{table_name} ADD COLUMN {column_name} {data_type}")
          except psycopg2.errors.DuplicateColumn:
            pass
          conn.commit()
          break
  conn.commit()



# Print the Schemas, Tables and Columns of the Assests

In [ ]:
# Get a list of all schemas in the current database starting with 'TEST_'
cur.execute("SELECT schema_name FROM information_schema.schemata WHERE schema_name LIKE 'TEST_%'")
schemas = cur.fetchall()

# Iterate over the schemas and print the name of each one
for schema in schemas:
  print('\n', "SCHEMA: ", schema[0])
  # Get a list of all tables in the current schema
  cur.execute(f"SELECT table_name FROM information_schema.tables WHERE table_schema = '{schema[0]}'")
  tables = cur.fetchall()
  # Iterate over the tables and print the name of each one
  for table in tables:
      print('\n', "TABLE: ", f"  {table[0]}", '\n', "COLUMNS: ")
      # Get a list of all columns in the current table
      cur.execute(f"SELECT column_name FROM information_schema.columns WHERE table_schema = '{schema[0]}' AND table_name = '{table[0]}'")
      columns = cur.fetchall()
      # Iterate over the columns and print the name of each one
      for column in columns:
          print(f"    {column[0]}")


 SCHEMA:  TEST_BTC-USD

 TABLE:    table_view_list 
 COLUMNS: 
    id

 TABLE:    table_view_column_list 
 COLUMNS: 
    id

 TABLE:    raw_features_data 
 COLUMNS: 
    id
    raw_features_data_pk
    raw_features_data_id
    raw_features_data_uuid
    raw_features_data_dcp
    raw_features_data_opcp
    raw_features_data_hpcp
    raw_features_data_lpc
    raw_features_data_cpcp
    raw_features_data_acpcp
    raw_features_data_vtcp
    raw_features_data_creator_pk
    raw_features_data_creation_date_time
    raw_features_data_last_modifier_pk
    raw_features_data_last_modification_date_time
    raw_features_data_activity_status_pk

 TABLE:    cleaned_raw_features_data 
 COLUMNS: 
    id
    cleaned_raw_features_data_pk
    cleaned_raw_features_data_id
    cleaned_raw_features_data_uuid
    cleaned_raw_features_data_dcp
    cleaned_raw_features_data_opcp
    cleaned_raw_features_data_hpcp
    cleaned_raw_features_data_lpcp
    cleaned_raw_features_data_cpcp
    cleaned_raw_features_

# Delete unwanted schema



In [ ]:
# Delete schema and it's content
for schema in schemas:
  schema_name = schema[0]
  print("Deleted schema: ", schema_name)
  # cur.execute(f"DROP SCHEMA {schema_name} CASCADE") # uncomment this line to delete to delete test shemas

Deleted schema:  TEST_BTC-USD
Deleted schema:  TEST_AAPL
Deleted schema:  TEST_^GSPC
Deleted schema:  TEST_MSFT
Deleted schema:  TEST_ZW=F
Deleted schema:  TEST_VUSA.L
Deleted schema:  TEST_HPQ
Deleted schema:  TEST_GC=F
Deleted schema:  TEST_000300.SS
Deleted schema:  TEST_HG=F
Deleted schema:  TEST_EURUSD=X
Deleted schema:  TEST_F
Deleted schema:  TEST_TSLA
Deleted schema:  TEST_EUEA.AS
Deleted schema:  TEST_CS.PA
Deleted schema:  TEST_GOOG


# Update Table Values

In [ ]:

# Update the values in the example_table
# cur.execute("UPDATE example_schema.example_table SET column1 = 'new_value' WHERE id = 1;")

# Commit the changes to the database
# conn.commit()

# Close the connection
# conn.close()